In [325]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [326]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
%matplotlib inline

In [327]:
df = pd.read_pickle('/Users/andrewpeters/GitHub/fpl/data/interim/01a-understat-merged-df.pkl')

In [328]:
len(df)

105230

In [329]:
df[['key_passes','xA', 'xGChain', 'xGBuildup']] = df[['key_passes','xA', 'xGChain', 'xGBuildup']].apply(pd.to_numeric)

To start, the main way I'll be adding features here is by making lag and rolling features. I also want to correct for that fact that not all data in each row is knowable _before_ the game. I know the fixture and the opponent before a game, but I don't know the minutes a player played until after, for example

In [330]:
prior_knowledge = ['player', 'team', 'position', 'gw', 'opponent_team', 'was_home', 'date', 'season']
posterior_knowledge = ['minutes', 'goals_scored', 'assists', 'clean_sheets', 
                       'goals_conceded', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index',
                      'transfers_balance', 'selected', 'transfers_in', 'transfers_out', 'key_passes',
                       'xA', 'xGChain', 'xGBuildup']

In [331]:
#ordering the columns so I can keep better track of this
#minutes are very much posterior knowledge as well, but I want to use this later to drop players who we could have easily predicted weren't going to play

df = df[['total_points'] + prior_knowledge + posterior_knowledge]

In [332]:
df = df.sort_values(['player', 'date'])
shifted_df = df.groupby('player')[['total_points'] + posterior_knowledge].shift(1)
df = df.join(shifted_df, rsuffix='_prev').sort_values(['player', 'date'])

In [333]:
posterior_knowledge_shift = [f'{col}_prev' for col in posterior_knowledge]

In [334]:
#generate rolling features, and join back with the original df. Using a minimum period of 4 to balance precision and not ending up with too many NaNs.
df = df.sort_values(['player', 'date'])
for window_size in tqdm(np.arange(2,15,2)):
    rolling_df = df.groupby('player')[['total_points_prev'] + posterior_knowledge_shift].rolling(window=window_size, min_periods=1).mean().reset_index().set_index('level_1')
    df = df.join(rolling_df[['total_points_prev'] + posterior_knowledge_shift], rsuffix = f'_{window_size}')

100%|██████████| 7/7 [00:10<00:00,  1.51s/it]


In [335]:
# at this point, I should drop the posterior_knowledge features -- the features we don't actually know until after the end of the gw
# i want to at least leave the 'minutes' col intact for the time being, because I might use this later for filter, or making the target value 'pts/minute'
posterior_knowledge.pop(0) 

'minutes'

In [336]:
df = df.drop(columns=posterior_knowledge)

# Now, onto a similar process, but for team-level features

In [338]:
team_level = pd.read_pickle('/Users/andrewpeters/GitHub/understat/fixture_level_stats.pkl')

In [339]:
team_level = team_level.drop(columns = ['id', 'isResult', 'team_opp'])

In [340]:
team_level = team_level.sort_values(['team', 'datetime']).reset_index(drop=True)

In [341]:
shifted_team_level = team_level.groupby('team').shift(1).reset_index(drop=True)

In [342]:
team_level = team_level.join(shifted_team_level, rsuffix='_prev')

In [343]:
team_level = team_level.drop(columns=['xg_team', 'xg_opp', 'team_goals', 'goals_opp', 'xg_diff', 'goal_diff', 'datetime_prev'])

In [344]:
team_level_posterior_cols = ['xg_team_prev', 'xg_opp_prev', 'team_goals_prev', 'goals_opp_prev', 'xg_diff_prev', 'goal_diff_prev']

In [345]:
team_level = team_level.sort_values(['team', 'datetime'])
for window_size in tqdm(np.arange(2,15,2)):
    rolling_team_df = team_level.groupby('team')[team_level_posterior_cols].rolling(window=window_size, min_periods=1).mean().reset_index().set_index('level_1')
    team_level = team_level.join(rolling_team_df[team_level_posterior_cols], rsuffix = f'_{window_size}')

100%|██████████| 7/7 [00:00<00:00, 41.01it/s]


In [346]:
team_level['date'] = team_level['datetime'].dt.date

In [347]:
team_level = team_level.drop(columns = 'datetime')

In [359]:
team_level = team_level.replace({'Brighton': 'Brighton and Hove Albion',
'West Ham': 'West Ham United',
'Huddersfield': 'Huddersfield Town',
'Sheffield United': 'Sheffield Utd',
 'Manchester United': 'Man Utd',
'Hull': 'Hull City',
'Wolverhampton Wanderers': 'Wolves',
'Leicester': 'Leicester City',
'Manchester City': 'Man City',
'West Bromwich Albion': 'West Brom',
 'Tottenham': 'Tottenham Hotspur',
 'Cardiff': 'Cardiff City',
 'Swansea': 'Swansea City',
 'Newcastle United': 'Newcastle',
 'Stoke' : 'Stoke City'})

In [361]:
df = df.replace({'Spurs': 'Tottenham Hotspur'})

In [305]:
#save the team_level table so I can use later for forward-looking games
team_level.to_pickle('/Users/andrewpeters/GitHub/fpl/data/interim/team_level_rolling.pkl')

In [351]:
#merge team level rolling stats
df = df.merge(team_level, left_on = ['team', 'date'], right_on = ['team', 'date'], suffixes = ('', '_team'), how = 'left')

In [352]:
#do the same for opponents, so this can provide features on the difficulty of the opponent
df = df.merge(team_level, left_on = ['opponent_team', 'date'], right_on = ['team', 'date'], suffixes = ('', '_opponent'), how = 'left')

In [353]:
df.to_pickle('/Users/andrewpeters/GitHub/fpl/data/interim/df_with_rolling_features.pkl')

In [354]:
team_level.team.unique()

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brighton', 'Burnley',
       'Cardiff', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham',
       'Huddersfield', 'Hull', 'Leeds', 'Leicester', 'Liverpool',
       'Manchester City', 'Manchester United', 'Middlesbrough',
       'Newcastle United', 'Norwich', 'Sheffield United', 'Southampton',
       'Stoke', 'Sunderland', 'Swansea', 'Tottenham', 'Watford',
       'West Bromwich Albion', 'West Ham', 'Wolverhampton Wanderers'],
      dtype=object)

In [355]:
df.team.unique()

array(['Brighton and Hove Albion', 'Brighton', 'West Ham United',
       'West Ham', 'Everton', 'Burnley', 'Huddersfield Town',
       'Bournemouth', 'Sheffield Utd', 'Arsenal', 'Crystal Palace',
       'Manchester United', 'Man Utd', 'Watford', 'Chelsea', 'Hull City',
       'Fulham', 'Middlesbrough', 'Leeds', 'Norwich', 'Liverpool',
       'Sunderland', 'Wolverhampton Wanderers', 'Wolves',
       'Leicester City', 'Leicester', 'Man City', 'West Bromwich Albion',
       'West Brom', 'Aston Villa', 'Manchester City', 'Newcastle United',
       'Southampton', 'Tottenham Hotspur', 'Cardiff City', 'Spurs',
       'Swansea City', 'Newcastle', 'Sheffield United', 'Stoke City'],
      dtype=object)

In [362]:
[team for team in df.team.unique() if team not in team_level.team.unique()]

['Brighton',
 'West Ham',
 'Manchester United',
 'Wolverhampton Wanderers',
 'Leicester',
 'West Bromwich Albion',
 'Manchester City',
 'Newcastle United',
 'Sheffield United']